In [ ]:
import os
import sys
import subprocess
import hail as hl
from pyspark.sql import SparkSession

In [ ]:
builder = (
    SparkSession
    .builder
    .enableHiveSupport())
spark = builder.getOrCreate()
hl.init(sc=spark.sparkContext)
hl.default_reference("GRCh38")
print("Hail version:", hl.__version__)

In [ ]:
!hdfs dfs -mkdir -p /tmp/pca_bgen
!hdfs dfs -put -f /mnt/project/dcm_pgs/pca_variants/dcm_pca_chr*_subset.bgen /tmp/pca_bgen/
!hdfs dfs -ls /tmp/pca_bgen | head -n 10

In [ ]:
contig_recoding = {f"{i:02d}": str(i) for i in range(1, 23)}

for c in range(1, 23):
    bgen = f"hdfs://master:9000/tmp/pca_bgen/dcm_pca_chr{c}_subset.bgen"
    idx2 = bgen + ".idx2"

    print(f"chr{c}: indexing")
    hl.index_bgen(
        path=bgen,
        index_file_map={bgen: idx2},
        reference_genome="GRCh37",
        contig_recoding=contig_recoding,
    )

print("Done indexing.")

In [ ]:
# BGEN files
bgen_paths = [
    f"hdfs://master:9000/tmp/pca_bgen/dcm_pca_chr{c}_subset.bgen"
    for c in range(1, 23)
]

mt = hl.import_bgen(
    bgen_paths,
    sample_file="file:///mnt/project/Bulk/Imputation/UKB imputation from genotype/ukb22828_c1_b0_v3.sample",
    entry_fields=["dosage"]
)

In [ ]:
# Annotate rows with loadings and filter to intersection
ld = hl.read_table("file:///mnt/project/dcm_pgs/loadings/gnomad.v3.1.pca_loadings_grch37.ht")
ld = ld.key_by("locus", "alleles")
mt = mt.annotate_rows(l = ld[mt.row_key])
mt = mt.filter_rows(hl.is_defined(mt.l))

In [ ]:
p = mt.l.pca_af
mu = 2.0 * p
sigma = hl.sqrt(2.0 * p * (1.0 - p))
x = (hl.float64(mt.dosage) - mu) / sigma

# compute PC1..PC10 without building a vector per entry
mt = mt.annotate_entries(
    c1 = x * mt.l.loadings[0],
    c2 = x * mt.l.loadings[1],
    c3 = x * mt.l.loadings[2],
    c4 = x * mt.l.loadings[3],
    c5 = x * mt.l.loadings[4],
    c6 = x * mt.l.loadings[5],
    c7 = x * mt.l.loadings[6],
    c8 = x * mt.l.loadings[7],
    c9 = x * mt.l.loadings[8],
    c10 = x * mt.l.loadings[9],
)

mt = mt.annotate_cols(
    PC1 = hl.agg.sum(mt.c1),
    PC2 = hl.agg.sum(mt.c2),
    PC3 = hl.agg.sum(mt.c3),
    PC4 = hl.agg.sum(mt.c4),
    PC5 = hl.agg.sum(mt.c5),
    PC6 = hl.agg.sum(mt.c6),
    PC7 = hl.agg.sum(mt.c7),
    PC8 = hl.agg.sum(mt.c8),
    PC9 = hl.agg.sum(mt.c9),
    PC10 = hl.agg.sum(mt.c10),
)
mt = mt.select_entries()

In [ ]:
pcs = mt.cols().select("PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10")

In [ ]:
# export sharded (prevents executor OOM)
pcs.export("ukb_gnomad_projected_pcs.tsv.bgz", parallel="header_per_shard")

# copy to project
hl.hadoop_copy(
    "ukb_gnomad_projected_pcs.tsv.bgz",
    "file:///mnt/project/dcm_pgs/ukb_gnomad_projected_pcs.tsv.bgz"
)